In [86]:
import numpy as np
import polars as pl

from sklearn.model_selection import GridSearchCV, KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [87]:
summary_data = pl.read_csv("water_quality_and_parcel_summaries_2004_to_2015.csv")
summary_data


,Monit_MAP_CODE1,Year,LAKE_NAME,Mean_Secchi_Depth_Result,Mean_Phosporus_Result,Mean_EMV_Total,STD_EMV_Total,Mean_Sale_Value,STD_Sale_Value,Mean_Total_Tax,STD_Total_Tax,Mean_Garage_Size,STD_Garage_Size,Mean_Fin_SQ_FT,STD_Fin_SQ_FT,Percentage_Yes_Basement,Percentage_Yes_Garage,Percentage_Yes_Tax_Exempt,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,Percentage_Air_Heating,Percentage_Space_Heating,Percentage_Water_Heating,Percentage_Electric_Heating,Percentage_Other_Heating,Percentage_No_Heating
i64,str,i64,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0,"""02000500-01""",2014,"""George Watch Lake""",0.716667,0.108778,215637.592745,459023.922281,122485.615829,183414.899771,3313.685903,17837.814516,null,null,1849.473207,8220.487298,70.486397,0.0,7.914262,0.0,0.0,0.0,2.308326,97.691674,0.0,0.0,0.0,0.0,85.325639,14.674361
1,"""02000500-01""",2013,"""George Watch Lake""",0.365,0.3105,196764.633141,454192.377508,120601.723001,190129.461399,3319.081616,18199.952917,null,null,1841.112119,8221.467234,69.991756,0.0,7.914262,0.824402,0.0,0.0,1.154163,0.0,79.060181,0.824402,1.154163,2.802968,1.154163,0.0
2,"""02000500-01""",2012,"""George Watch Lake""",0.359,0.2649,200414.333057,480355.968668,118315.128418,190091.029827,3460.064623,19346.923479,null,null,1811.576636,8223.512958,70.173985,0.0,8.119304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,"""02000500-01""",2011,"""George Watch Lake""",0.973333,0.119417,216297.932175,536941.029067,111218.812242,190312.451914,3459.313482,19320.28833,null,null,1780.282051,8215.092878,76.840364,0.0,8.271299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"""02000500-01""",2010,"""George Watch Lake""",0.493333,0.173,222789.430223,559656.083477,110765.119736,190448.272924,3384.645747,18646.626107,null,null,1760.876135,8210.519477,75.887696,0.0,8.092486,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
523,"""82036800-01""",2008,"""Klawitter Pond""",0.596923,0.092462,420461.350844,802637.556411,128391.380863,182645.51138,412444.840525,798052.142235,null,null,0.0,0.0,0.0,0.0,1.876173,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
524,"""82036800-01""",2007,"""Klawitter Pond""",0.491667,0.098083,396113.320826,487241.389735,131310.281426,178831.780564,0.0,0.0,null,null,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
525,"""82036800-01""",2006,"""Klawitter Pond""",0.6,0.1035,371583.114447,261100.184471,120477.452158,168187.904304,2743.87242,1971.437642,null,null,1634.529081,1160.042283,0.0,0.0,1.125704,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Variables and Feature Columns

In [88]:
SECCHI_COL = "Mean_Secchi_Depth_Result"
TP_COL = "Mean_Phosporus_Result"

id_cols = ["Monit_MAP_CODE1", "LAKE_NAME", "Year", "Unnamed: 0"]

feature_cols = [
    c for c in summary_data.columns
    if c not in id_cols + [SECCHI_COL, TP_COL]
]

len(feature_cols), feature_cols[:5]


(25,
 ['', 'Mean_EMV_Total', 'STD_EMV_Total', 'Mean_Sale_Value', 'STD_Sale_Value'])

In [89]:
lakes = (
    summary_data
    .select("Monit_MAP_CODE1")
    .unique()
    .to_series()
    .to_list()
)

len(lakes), lakes[:5]


(48,
 ['27004201-01', '19002500-01', '19034800-01', '19002400-01', '82010400-01'])

### Train/Test/Validation Splits

In [90]:
np.random.seed(326)

validation_ids = np.random.choice(
    lakes,
    size = int(0.30 * len(lakes)),
    replace = False
)

remaining_ids = np.setdiff1d(lakes, validation_ids)

train_ids = np.random.choice(
    remaining_ids,
    size = int(0.70 * len(remaining_ids)),
    replace = False
)

test_ids = np.setdiff1d(remaining_ids, train_ids)

len(validation_ids), len(train_ids), len(test_ids)


(14, 23, 11)

In [91]:
data_labeled = (
    summary_data
    .with_columns(
        pl.when(pl.col("Monit_MAP_CODE1").is_in(validation_ids))
        .then(pl.lit("Validation"))
        .when(pl.col("Monit_MAP_CODE1").is_in(train_ids))
        .then(pl.lit("Training"))
        .otherwise(pl.lit("Test"))
        .alias("TrainTestValid")
    )
)

data_labeled.select(
    pl.col("TrainTestValid").value_counts()
)


TrainTestValid
struct[2]
"{""Test"",121}"
"{""Training"",253}"
"{""Validation"",154}"


### Splitting Data

In [92]:
train_data = data_labeled.filter(pl.col("TrainTestValid") == "Training")
test_data = data_labeled.filter(pl.col("TrainTestValid") == "Test")
valid_data = data_labeled.filter(pl.col("TrainTestValid") == "Validation")

len(train_data), len(test_data), len(valid_data)


(253, 121, 154)

### Secchi Depth Metrics

In [93]:
X_train_secchi = train_data.select(feature_cols).to_numpy()
y_train_secchi = train_data.select(SECCHI_COL).to_numpy().ravel()

X_test_secchi = test_data.select(feature_cols).to_numpy()
y_test_secchi = test_data.select(SECCHI_COL).to_numpy().ravel()

X_val_secchi = valid_data.select(feature_cols).to_numpy()
y_val_secchi = valid_data.select(SECCHI_COL).to_numpy().ravel()


### CART and RF - Secchi

In [94]:
cart_params = {
    "max_depth": [3, 5, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
}

rf_params = {
    "n_estimators": [100, 200, 300],
    "max_depth": [5, 10, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 5],
    "max_features": ["sqrt", "log2"],
}

cv_obj = KFold(n_splits=10, shuffle=True, random_state=326)


### Grid Search - Secchi

In [95]:
grid_cart_secchi = GridSearchCV(
    DecisionTreeRegressor(random_state=326),
    cart_params,
    cv=cv_obj,
    scoring="r2",
    n_jobs=-1,
    verbose=0,
)

grid_cart_secchi.fit(X_train_secchi, y_train_secchi)




,estimator,DecisionTreeR...dom_state=326)
,param_grid,"{'max_depth': [3, 5, ...], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 5, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [96]:
grid_cart_secchi.best_score_, grid_cart_secchi.best_params_

(np.float64(0.6117315243427933),
 {'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2})

In [97]:
best_cart_secchi = grid_cart_secchi.best_estimator_
best_cart_secchi


,criterion,'squared_error'
,splitter,'best'
,max_depth,5
,min_samples_split,2
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,326
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


### RF - Secchi

In [98]:
grid_rf_secchi = GridSearchCV(
    estimator=RandomForestRegressor(random_state=326),
    param_grid=param_grid_rf,
    cv=cv_obj,
    scoring="r2",
    n_jobs=-1,
    verbose=0,
)

grid_rf_secchi.fit(X_train_secchi, y_train_secchi)


,estimator,RandomForestR...dom_state=326)
,param_grid,"{'max_depth': [5, 10, ...], 'min_samples_leaf': [1, 2], 'min_samples_split': [2, 5], 'n_estimators': [100, 200, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,300


In [99]:
grid_rf_secchi.best_score_, grid_rf_secchi.best_params_


(np.float64(0.7466273759918298),
 {'max_depth': 10,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 300})

In [100]:
best_rf_secchi = grid_rf_secchi.best_estimator_
best_rf_secchi


,n_estimators,300
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [101]:
cart_test_pred_secchi = best_cart_secchi.predict(X_test_secchi)
rf_test_pred_secchi = best_rf_secchi.predict(X_test_secchi)

cart_r2_secchi = r2_score(y_test_secchi, cart_test_pred_secchi)
rf_r2_secchi = r2_score(y_test_secchi, rf_test_pred_secchi)

cart_rmse_secchi = np.sqrt(mean_squared_error(y_test_secchi, cart_test_pred_secchi))
rf_rmse_secchi = np.sqrt(mean_squared_error(y_test_secchi, rf_test_pred_secchi))

cart_mae_secchi = mean_absolute_error(y_test_secchi, cart_test_pred_secchi)
rf_mae_secchi = mean_absolute_error(y_test_secchi, rf_test_pred_secchi)

cart_r2_secchi, rf_r2_secchi, cart_rmse_secchi, rf_rmse_secchi, cart_mae_secchi, rf_mae_secchi


(-0.12111776744801195,
 0.009879048970639404,
 np.float64(1.3312914333262744),
 np.float64(1.2510989076102152),
 0.9742783322917493,
 1.0013109827501858)

### Best Secchi Model

In [102]:
best_model_secchi = best_rf_secchi if rf_r2_secchi >= cart_r2_secchi else best_cart_secchi
best_model_secchi


,n_estimators,300
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [104]:
drop_cols = [
    "Mean_Secchi_Depth_Result",
    "Mean_Phosporus_Result",
    "TrainTestValid",
    "Monit_MAP_CODE1",
    "LAKE_NAME",
    "Year"
]


In [105]:
final_train_secchi = train_data.vstack(test_data)

final_X_secchi = final_train_secchi.drop(drop_cols).to_pandas()
final_y_secchi = final_train_secchi.select(SECCHI_COL).to_pandas().iloc[:, 0]

best_model_secchi_final = best_model_secchi.fit(final_X_secchi, final_y_secchi)


### Validation

In [107]:
X_val_secchi = valid_data.drop(drop_cols).to_pandas()
y_val_secchi = valid_data.select(SECCHI_COL).to_pandas().iloc[:, 0]

y_val_pred_secchi = best_model_secchi_final.predict(X_val_secchi)

val_r2_secchi = r2_score(y_val_secchi, y_val_pred_secchi)
val_rmse_secchi = np.sqrt(mean_squared_error(y_val_secchi, y_val_pred_secchi))
val_mae_secchi = mean_absolute_error(y_val_secchi, y_val_pred_secchi)

val_r2_secchi, val_rmse_secchi, val_mae_secchi


(-0.3448601173847132, np.float64(1.093661757246699), 0.885725457690587)

In [109]:
param_grid_cart = {
    "max_depth": [None, 2, 3, 4, 5],
    "min_samples_split": [2, 3, 5, 10],
    "min_samples_leaf": [1, 5, 10],
}


### TP Metrics

In [111]:
X_train_tp = train_data.drop(drop_cols).to_pandas()
y_train_tp = train_data.select(TP_COL).to_pandas().iloc[:, 0]

X_val_tp = valid_data.drop(drop_cols).to_pandas()
y_val_tp = valid_data.select(TP_COL).to_pandas().iloc[:, 0]

X_test_tp = test_data.drop(drop_cols).to_pandas()
y_test_tp = test_data.select(TP_COL).to_pandas().iloc[:, 0]


### Grid Search - TP

In [112]:
grid_cart_tp = GridSearchCV(
    estimator=DecisionTreeRegressor(),
    param_grid=param_grid_cart,
    cv=cv_obj,
    scoring="r2",
    n_jobs=-1,
    verbose=0,
)

grid_cart_tp.fit(X_train_tp, y_train_tp)


,estimator,DecisionTreeRegressor()
,param_grid,"{'max_depth': [None, 2, ...], 'min_samples_leaf': [1, 5, ...], 'min_samples_split': [2, 3, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,criterion,'squared_error'


In [113]:
grid_cart_tp.best_score_, grid_cart_tp.best_params_


(np.float64(0.32739956587407404),
 {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 3})

In [114]:
best_cart_tp = grid_cart_tp.best_estimator_
best_cart_tp


,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,3
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


In [115]:
grid_rf_tp = GridSearchCV(
    estimator=RandomForestRegressor(random_state=326),
    param_grid=param_grid_rf,
    cv=cv_obj,
    scoring="r2",
    n_jobs=-1,
    verbose=0,
)

grid_rf_tp.fit(X_train_tp, y_train_tp)


,estimator,RandomForestR...dom_state=326)
,param_grid,"{'max_depth': [5, 10, ...], 'min_samples_leaf': [1, 2], 'min_samples_split': [2, 5], 'n_estimators': [100, 200, ...]}"
,scoring,'r2'
,n_jobs,-1
,refit,True
,cv,KFold(n_split... shuffle=True)
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,300


In [116]:
grid_rf_tp.best_score_, grid_rf_tp.best_params_


(np.float64(0.5565705372336113),
 {'max_depth': None,
  'min_samples_leaf': 1,
  'min_samples_split': 2,
  'n_estimators': 300})

In [117]:
best_rf_tp = grid_rf_tp.best_estimator_
best_rf_tp


,n_estimators,300
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


### CART / RF - TP

In [118]:
cart_test_pred_tp = best_cart_tp.predict(X_test_tp)
rf_test_pred_tp = best_rf_tp.predict(X_test_tp)

cart_r2_tp = r2_score(y_test_tp, cart_test_pred_tp)
rf_r2_tp = r2_score(y_test_tp, rf_test_pred_tp)

cart_rmse_tp = np.sqrt(mean_squared_error(y_test_tp, cart_test_pred_tp))
rf_rmse_tp = np.sqrt(mean_squared_error(y_test_tp, rf_test_pred_tp))

cart_mae_tp = mean_absolute_error(y_test_tp, cart_test_pred_tp)
rf_mae_tp = mean_absolute_error(y_test_tp, rf_test_pred_tp)

cart_r2_tp, rf_r2_tp, cart_rmse_tp, rf_rmse_tp, cart_mae_tp, rf_mae_tp


(-0.23229004341685888,
 -0.2834518511385369,
 np.float64(0.06778568902319379),
 np.float64(0.06917853098716542),
 0.05142560435394349,
 0.05360977907421859)

### Best Model

In [119]:
best_model_tp = best_rf_tp if rf_r2_tp >= cart_r2_tp else best_cart_tp
best_model_tp


,criterion,'squared_error'
,splitter,'best'
,max_depth,None
,min_samples_split,3
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,None
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,ccp_alpha,0.0


### Refitting Model

In [120]:
final_train_tp = train_data.vstack(test_data)

final_X_tp = final_train_tp.drop(drop_cols).to_pandas()
final_y_tp = final_train_tp.select(TP_COL).to_pandas().iloc[:, 0]

best_model_tp_final = best_model_tp.fit(final_X_tp, final_y_tp)


In [121]:
X_val_tp = valid_data.drop(drop_cols).to_pandas()
y_val_tp = valid_data.select(TP_COL).to_pandas().iloc[:, 0]

y_val_pred_tp = best_model_tp_final.predict(X_val_tp)

val_r2_tp = r2_score(y_val_tp, y_val_pred_tp)
val_rmse_tp = np.sqrt(mean_squared_error(y_val_tp, y_val_pred_tp))
val_mae_tp = mean_absolute_error(y_val_tp, y_val_pred_tp)

val_r2_tp, val_rmse_tp, val_mae_tp


(-0.637857007595205, np.float64(0.10420640568852266), 0.07281114883836703)

Random Forest performed better than CART for both Secchi Depth and Total Phosphorus, so I picked it to use for my final model. 
However, after I refitted on the full 70% (training + test) data and evaluated it on the validation lakes, the results were very poor ... I was only able to get
negative R^2, which is not helpful. 

Secchi Depth: R^2 = –0.345

Total Phosphorus: R^2 = –0.638

These R^2 values showed me that the models don't fit to new lakes. In other words, parcel-level variables simply do not work well across lakes, and they are not effective predictors of lake water quality.